In [1]:
import torch
from lstm_models import UtilizationLSTM

/Users/macbook/opt/miniconda3/envs/ml_pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_parameters = torch.load('models/small-epochs-100', map_location=torch.device('cpu'))

In [3]:
model_parameters.keys()

dict_keys(['epoch', 'learning_rate', 'input_size', 'hidden_size', 'num_layers', 'num_classes', 'model_state_dict', 'optimizer_state_dict'])

In [4]:
num_classes = model_parameters['num_classes']
input_size = model_parameters['input_size']
hidden_size = model_parameters['hidden_size']
num_layers = model_parameters['num_layers']

In [5]:
lstm_model = UtilizationLSTM(input_size, hidden_size, num_layers)

In [6]:
lstm_model.load_state_dict(model_parameters['model_state_dict'])

<All keys matched successfully>

In [7]:
lstm_model.eval()

UtilizationLSTM(
  (cpu_lstm): LSTM(4, 4000, num_layers=2, batch_first=True)
  (mem_lstm): LSTM(4, 4000, num_layers=2, batch_first=True)
  (cpu_lstm_seq): Sequential(
    (0): ReLU()
    (1): BatchNorm1d(4000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=4000, out_features=512, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_features=512, out_features=128, bias=True)
    (6): ReLU()
    (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Linear(in_features=128, out_features=1, bias=True)
  )
  (mem_lstm_seq): Sequential(
    (0): ReLU()
    (1): BatchNorm1d(4000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=4000, out_features=512, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_

In [13]:
test_tensor = torch.ones([300, 1, 4], dtype=torch.float32)
    # test_tensor = torch.ones([1000, 1, 8], dtype=torch.float32).to(device)
    
for i in range(test_tensor.size(2)):
    test_tensor[:, :, i] *= test_tensor[:, :, i] * (i)

# lstm_model(test_tensor)

In [106]:
resources = [
    [80, 100, 0.5, 1],
    [90, 100, 1, 1],
    [50, 50, 0.5, 0.5]
    ]
target = [
    [75, 0.7],
    [90, 0.75],
    [35, 0.30],
    
]

In [107]:
from dataframe_scaler import DataFrameScaler
import pandas as pd

In [108]:
df = pd.DataFrame(resources)
dataframe_scaler = DataFrameScaler(df)
std_df = dataframe_scaler.standardize_df(df)
std_df

,0,1,2,3
0,0.320256,0.577350,-0.577350,0.577350
1,0.800641,0.577350,1.154701,0.577350
2,-1.120897,-1.154701,-0.577350,-1.154701


In [109]:
target_df = pd.DataFrame(target)
target_scaler = DataFrameScaler(target_df)
norm_df = target_scaler.normalize_df(target_df)
norm_df

,0,1
0,0.727273,0.888889
1,1.000000,1.000000
2,0.000000,0.000000


In [110]:
std_tensor = torch.FloatTensor(std_df.values)
std_tensor.shape

torch.Size([3, 4])

In [111]:
# resource_tensor = resource_tensor.reshape(resource_tensor.size(0), 1, resource_tensor.size(1))
std_tensor = std_tensor.reshape(std_tensor.size(0), 1, std_tensor.size(1))

In [112]:
prediction = lstm_model(std_tensor)
prediction

tensor([[0.0435, 0.0102],
        [0.0099, 0.0233],
        [0.1254, 0.1232]], grad_fn=<AbsBackward0>)

In [113]:
prediction_df = pd.DataFrame(prediction.data.numpy())
target_scaler.inverse_normalization_df(prediction_df)

,0,1
0,37,0.304573
1,35,0.310478
2,41,0.355425


In [114]:
prediction.shape

torch.Size([3, 2])